## Introduction: Business Problem <a name="introduction"></a>

These days we deal with coronavirus and fast intervention is quite important. We want to show the nearest hospital to a coronavirus patient so in this way they can intervene faster to the patient. And prevent to spread it much more because of that patient.

## Data <a name="data"></a>

In introduction section we specified the problem that we will solve. The problem was to find the coronavirus patients' location first. After finding the patients, showing them the nearest hospital to go. So for this we need patients' locations and hospitals' locations. We can get hospitals' locations from Foursquare location data but there is no data to find where the coronavirus patents are. And this is not too important because I will specify some locations for the patients in different places. In this way, we can find the hospitals near to the patient and show the nearest one.

### Near Hospitals

Firstly there are some libraries we need to import, we import them. After this we need a Client Id and Client Secret to connect and to use the Foursquare location system. We use geolocator to find a city or country's location by it's name.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
CLIENT_ID = 'ABM5N3GPUYTNFBPUHTD1OLKSIRVCR5G3RMX1AG3YC5WYMJ43' # your Foursquare ID
CLIENT_SECRET = 'HSOVATVLZNX5OEW04EO1TACWEGOQ3BFOKUHSG4VI0FEITNOA' # your Foursquare Secret

In [3]:
address = 'Istanbul'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

folium.Map(location=[41.0096334, 28.9651646],zoom_start=10)

41.0096334 28.9651646


## Methodology <a name="methodology"></a>

We need patient's location. After that we find the hospitals that near to the patient. Foursquare gives us the distance too. So this makes us faster. We sort distance and we show the nearest hospital to patient as a black icon while the others' colors were blue. And we print the nearest hospital's name.

### Analysis

The location of coronavirus patient will be unique, this will be our location, because of that I created a new dataframe named df as patients's locations.

In [4]:
# Creating the dataframe that contains coronavirus patients' location.
df = pd.DataFrame({"Latitude":[40.955187,40.9670336,41.0138276,41.0450986,41.0071775],
     "Longitude":[29.300388,29.054685,29.045683,28.9712634,28.9735483]})

There is a point we need to notice, we create a URL to get the data from Foursquare. There is categoryId in the URL, this specifies the category that hospital, or cafe or restaurant etc. And you can reach them from here : https://developer.foursquare.com/docs/build-with-foursquare/categories

In [5]:
LATITUDE = df["Latitude"][0]
LONGITUDE = df["Longitude"][0]
RADIUS = 10000
LIMIT = 20
URL = "https://api.foursquare.com/v2/venues/search?ll={},{}&categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&radius={}&limit={}&v=20180628".format(LATITUDE,LONGITUDE,CLIENT_ID,CLIENT_SECRET,RADIUS,LIMIT)
URL

'https://api.foursquare.com/v2/venues/search?ll=40.955187,29.300388&categoryId=4bf58dd8d48988d196941735&client_id=ABM5N3GPUYTNFBPUHTD1OLKSIRVCR5G3RMX1AG3YC5WYMJ43&client_secret=HSOVATVLZNX5OEW04EO1TACWEGOQ3BFOKUHSG4VI0FEITNOA&radius=10000&limit=20&v=20180628'

Assign the data that URL contains.

In [6]:
results = requests.get(URL).json()

We create a dataframe that contains hospital name, latitude, longitude and the distance to the patient of the hospital.

In [7]:
samples = {}
for index,venue in enumerate(results["response"]["venues"]):
    samples.update({index:{"Hospital Name":venue["name"],
        "Latitude":venue["location"]["lat"],
        "Longitude":venue["location"]["lng"],
        "Distance":venue["location"]["distance"]}})
df_near_hospitals = pd.DataFrame(samples).T.sort_values(by="Distance")[:20]
df_near_hospitals.index = range(len(df_near_hospitals))

In [8]:
df_near_hospitals

,Hospital Name,Latitude,Longitude,Distance
0,Ersoy Hastanesi,40.9665,29.2694,2892
1,Özel Adatıp Hospital,40.9258,29.2912,3367
2,Emsey Hospital,40.924,29.2893,3599
3,Sultanbeyli Yeni Devlet Hastanesi,40.9862,29.2826,3764
4,Özel Saygı Hastanesi,40.9697,29.2578,3930
5,Ersoy Hastanesi,40.9109,29.2982,4938
6,Marmara Üniversitesi Pendik Eğitim Ve Araştırm...,40.8896,29.2651,7878
7,Marmara Üniversitesi Pendik Eğitim ve Araştırm...,40.8865,29.263,8268
8,Cihan Tıp Merkezi,40.8788,29.2716,8839
9,Sancaktepe Sahra Hastanesi,40.998,29.2115,8860


Lastly here we show the nearest hospital to the patient.

In [9]:
istanbul_map = folium.Map(location=[41.0096334, 28.9651646],zoom_start=10)

"""folium.features.CircleMarker(
[df["Latitude"][0],df["Longitude"][0]],
radius=5,
popup="YOU ARE HERE!",
fill=True,
color="red",
fill_color="red",
fill_opacity=0.6).add_to(istanbul_map)"""

folium.Marker(
location=[df["Latitude"][0],df["Longitude"][0]],
popup="YOU ARE HERE!",
icon=folium.Icon(color="red",
                icon="empty")).add_to(istanbul_map)


for latitude,longitude,label,distance in zip(df_near_hospitals["Latitude"],
                                   df_near_hospitals["Longitude"],
                                   df_near_hospitals["Hospital Name"],
                                   df_near_hospitals["Distance"]):
    if distance==min(df_near_hospitals["Distance"]):
        """folium.features.CircleMarker(
            [latitude,longitude],
            radius=7.5,
            popup=label,
            fill=True,
            color="black",
            fill_color="black",
            fill_opacity=1
        ).add_to(istanbul_map)"""
        folium.Marker(
        location=[latitude,longitude],
        popup=label,
        icon=folium.Icon(color="black",
                        icon="empty")).add_to(istanbul_map)
        
    else:
        folium.features.CircleMarker(
            [latitude,longitude],
            radius=5,
            popup=label,
            fill=True,
            color="blue",
            fill_color="blue",
            fill_opacity=1
        ).add_to(istanbul_map)
        
istanbul_map



print("{} is the nearest hospital to you!".format(df_near_hospitals[df_near_hospitals["Distance"]==min(df_near_hospitals["Distance"])]["Hospital Name"].values[0]))
istanbul_map

Ersoy Hastanesi is the nearest hospital to you!


## Results and Discussion <a name="results"></a>

By this way the patient can find the nearest hospital. This can be develop by making some visualizations or only giving location maybe we can show the patient directions or maybe we can send find the hospital's number and send the patient's location to them as a message etc.

## Conclusion <a name="conclusion"></a>

This project aimed that spend less time to search the nearest hospital. But as we say previous section this project can develop much much more. Thank you.